# Variables

In [1]:
source("/home/vickydiliscia/dmeyf2025/src/competencia/experimento/config.r")

## Semillas
PARAM <- list()
PARAM$semilla_primigenia <- SEEDS[1]
PARAM$experimento <- 5001
# training y future
PARAM$train <- c(202102)
PARAM$train_final <- c(202101,202102,202103,202104)
PARAM$future <- c(202104)
PARAM$semilla_kaggle <- 314159
PARAM$cortes <- seq(6000, 19000, by= 500)

# undersampling
PARAM$trainingstrategy$undersampling <- 0.3

# carpeta de trabajo
setwd("/content/buckets/b1/exp")
experimento_folder <- paste0("HT", PARAM$experimento)
dir.create(experimento_folder, showWarnings=FALSE)
setwd( paste0("/content/buckets/b1/exp/", experimento_folder ))

# Librerias

In [ ]:
# cargo las librerias que necesito
require("data.table")
require("parallel")

if(!require("R.utils")) install.packages("R.utils")
require("R.utils")

if( !require("primes") ) install.packages("primes")
require("primes")

if( !require("utils") ) install.packages("utils")
require("utils")

if( !require("rlist") ) install.packages("rlist")
require("rlist")

if( !require("yaml")) install.packages("yaml")
require("yaml")

if( !require("lightgbm") ) install.packages("lightgbm")
require("lightgbm")

if( !require("DiceKriging") ) install.packages("DiceKriging")
require("DiceKriging")

if( !require("mlrMBO") ) install.packages("mlrMBO")
require("mlrMBO")

In [ ]:
# Dataset

In [3]:
# lectura del dataset
dataset <- fread(DATASET_FEATURES, stringsAsFactors= TRUE)

In [ ]:
# Tomo hiperparametros

In [ ]:
kbayesiana <- "bayesiana.RDATA"  # tu archivo con el opt.state adentro
# Cargar el archivo (esto mete en memoria los objetos guardados, p. ej. opt.state)
load(kbayesiana)

# Si el archivo contiene el estado de la optimización:
bayesiana_salida <- list(opt.path = opt.state$opt.path)
                         
tb_bayesiana <- as.data.table(bayesiana_salida$opt.path)
colnames( tb_bayesiana)

# almaceno los resultados de la Bayesian Optimization
# y capturo los mejores hiperparametros encontrados

tb_bayesiana <- as.data.table(bayesiana_salida$opt.path)

tb_bayesiana[, iter := .I]

# ordeno en forma descendente por AUC = y
setorder(tb_bayesiana, -y)

# grabo para eventualmente poder utilizarlos en OTRA corrida
fwrite( tb_bayesiana,
  file= "BO_log.txt",
  sep= "\t"
)

# los mejores hiperparámetros son los que quedaron en el registro 1 de la tabla
PARAM$out$lgbm$mejores_hiperparametros <- tb_bayesiana[
  1, # el primero es el de mejor AUC
  setdiff(colnames(tb_bayesiana),
    c("y","dob","eol","error.message","exec.time","ei","error.model",
      "train.time","prop.type","propose.time","se","mean","iter")),
  with= FALSE
]

PARAM$out$lgbm$y <- tb_bayesiana[1, y]

write_yaml( PARAM, file="PARAM.yml")


print(PARAM$out$lgbm$mejores_hiperparametros)
print(PARAM$out$lgbm$y)